In [68]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.simplefilter('ignore')

- Submission_1 : Single XGB with random state 0 with SMOTE
- Submission_2 : Combination of 5 XGB and 5 Random Forest with Random state 0 with SMOTE
- Submission_3 : Combination of 5 Random Forest with Random state 0 with SMOTE
- Submission_4 : Submission 3 with Annual Income Dropped
- Submission_5 : Submission 2 with Annual Income Dropped

In [69]:
train = pd.read_csv('TrainingData.csv')
test = pd.read_csv('testX.csv')

df = pd.concat([train, test], axis = 0)

df.shape

(130000, 53)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130000 entries, 0 to 46999
Data columns (total 53 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   application_key  130000 non-null  int64  
 1   mvar1            130000 non-null  object 
 2   mvar2            120787 non-null  float64
 3   mvar3            129171 non-null  float64
 4   mvar4            129171 non-null  float64
 5   mvar5            129171 non-null  float64
 6   mvar6            130000 non-null  object 
 7   mvar7            130000 non-null  object 
 8   mvar8            130000 non-null  object 
 9   mvar9            130000 non-null  object 
 10  mvar10           130000 non-null  object 
 11  mvar11           130000 non-null  object 
 12  mvar12           130000 non-null  object 
 13  mvar13           130000 non-null  object 
 14  mvar14           130000 non-null  int64  
 15  mvar15           130000 non-null  object 
 16  mvar16           130000 non-null  objec

In [71]:
columns = ['ID',
           'Credit Worthiness Score',
           'Number and Riskiness of Credit Enquiry',
           'Default Severity (Any Loan)',
           'Default Severity (Auto Loan)',
           'Default Severity (Education Loan)',
           'Min Cred Available (Revolving Credit Cards)',
           'Max Cred Available (Active Credit Lines)',
           'Max Cred Available (Active Revolving Credit Cards)',
           'Total Available Credit (Cards with 1 missed payment)',
           'Total Credit Available',
           'Dues Collected Post Default',
           'Total Amount Due (Active Cards)',
           'Credit Amount Paid (Prev Year)',
           'Annual Income',
           'Property Value',
           'Full Credit Utilisation (Revolving Credit Cards)',
           'Full Credit Utilisation (Credit Cards)',
           'Full Credit Utilisation (Credit Lines)',
           '>75 percent Credit Utilisation (Credit Cards)',
           '>75 percent Credit Utilisation (Credit Lines)',
           'Average Utilisation (Revolving Credit Cards)',
           'Average Utilisation Last 2 Years (All Credit Lines)',
           'Average Utilisation Last Year (All Credit Cards)',
           'Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards)',
           'Average Tenure (Active Revolving Credit Cards)',
           'Tenure of Oldest Active Credit Card',
           'Tenure of Oldest Active Revolving Credit Card',
           'Days Since Last Missed Payment',
           'Tenure of Oldest Credit Line',
           'Max Tenure (Auto Loans)',
           'Max Tenure (Education Loans)',
           'Sum of Tenures (Active Credit Cards)',
           'Stay Duration of Current Residence',
           'Credit Lines with 1 Missed Payment (Last 6 Months)',
           'Revolving Credit Cards with 1 Missed Payment (Last 2 Years)',
           'Active Credit Lines',
           'Credit Cards with >2Y Tenure',
           'Credit Lines activated Prev 2 Years',
           'Credit Lines with Current Delinquency',
           'Utilization on Active Education Loans',
           'Utilization on Active Auto Loans',
           'Financial Stress Index',
           'Credit Lines with No Missed Payments (Prev 2 Years)',
           'Ratio(Max Amount Due : Sum of Amounts Due)',
           'Mortgage Loans (With 2 Missed Payements)',
           'Auto Loans (With 2 Missed Payments)',
           'Type of Product',
           'Int Value for an Application',
           'Bucketized Credit Worthiness Score',
           'Compound of mvar49 and mvar48',
           'mvar51',
           'default_ind']

In [72]:
df.columns = columns

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130000 entries, 0 to 46999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  int64  
 1   Credit Worthiness Score                                                 130000 non-null  object 
 2   Number and Riskiness of Credit Enquiry                                  120787 non-null  float64
 3   Default Severity (Any Loan)                                             129171 non-null  float64
 4   Default Severity (Auto Loan)                                            129171 non-null  float64
 5   Default Severity (Education Loan)                                       129171 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                          

### Feature Understanding (Explanations and Stuff to Look into)

- Auto Loans: Loans where Vehicles are kept as collateral
- Revolving Credit: You have a fixed amount that you can borrow in some fixed time frame. After that the amount you can borrow replenishes to the full amount. Although you will have a maximum limit. For example, you can borrow 100 rupees a month max and the overall limit on that credit line is 500 rupees. So if your overall borrowed amount is 460 rupees and for that month you havenn't borrowed anything so far, you will be able to borrow only 40 rupees even though the monthly limit is 100
- Difference between Credit Worthiness Score and its bucketized version?
- Difference in statistics between train and test (Need for Adverserial Validation?)

### Feature Ideas and Experiments
- Binary Variable for checking if the person has moved into current residence after issuing any credit lines
- Credit Amount Paid / Number of Credit Lines with No Missed Payments
- Max Cred Available vs Default Severity
- Separate modeling for 0 Default Severity vs Non-Zero Default Severity
- KNN Missing Values Fill In
- Property Value vs Annual Income (Some Indicator of whether or not a person spends too much for their Income)
- 75% utilisation / Full Utilisation

In [74]:
df.head(10)

,ID,Credit Worthiness Score,Number and Riskiness of Credit Enquiry,Default Severity (Any Loan),Default Severity (Auto Loan),Default Severity (Education Loan),Min Cred Available (Revolving Credit Cards),Max Cred Available (Active Credit Lines),Max Cred Available (Active Revolving Credit Cards),Total Available Credit (Cards with 1 missed payment),Total Credit Available,Dues Collected Post Default,Total Amount Due (Active Cards),Credit Amount Paid (Prev Year),Annual Income,Property Value,Full Credit Utilisation (Revolving Credit Cards),Full Credit Utilisation (Credit Cards),Full Credit Utilisation (Credit Lines),>75 percent Credit Utilisation (Credit Cards),>75 percent Credit Utilisation (Credit Lines),Average Utilisation (Revolving Credit Cards),Average Utilisation Last 2 Years (All Credit Lines),Average Utilisation Last Year (All Credit Cards),Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards),Average Tenure (Active Revolving Credit Cards),Tenure of Oldest Active Credit Card,Tenure of Oldest Active Revolving Credit Card,Days Since Last Missed Payment,Tenure of Oldest Credit Line,Max Tenure (Auto Loans),Max Tenure (Education Loans),Sum of Tenures (Active Credit Cards),Stay Duration of Current Residence,Credit Lines with 1 Missed Payment (Last 6 Months),Revolving Credit Cards with 1 Missed Payment (Last 2 Years),Active Credit Lines,Credit Cards with >2Y Tenure,Credit Lines activated Prev 2 Years,Credit Lines with Current Delinquency,Utilization on Active Education Loans,Utilization on Active Auto Loans,Financial Stress Index,Credit Lines with No Missed Payments (Prev 2 Years),Ratio(Max Amount Due : Sum of Amounts Due),Mortgage Loans (With 2 Missed Payements),Auto Loans (With 2 Missed Payments),Type of Product,Int Value for an Application,Bucketized Credit Worthiness Score,Compound of mvar49 and mvar48,mvar51,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,missing,6423,3067,123875,missing,1,1,1,3,3,94.78,8987.18,NaN,72.25,1462,4532,2890,61,4532,1095,3376,625,1.1667,1,0,8,10,4,1,73.78,82.547,0.08696,10,0.63899,na,0,C,10,770,4,3080,0.0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,missing,765,1931,42613,missing,0,0,0,0,0,74.25,953.06,953.06,4.80,1028,2099,2099,30386,2281,missing,2251,169,0.4167,0,0,8,0,2,0,99.129,missing,0,13,0.63836,na,na,L,732,437,5,2185,1.0
2,230034,1745,0.4001,0.000,0.0,0.0,missing,2536,missing,missing,2536,missing,missing,missing,76109,missing,na,na,na,0,0,NaN,NaN,NaN,NaN,missing,missing,missing,669,4623,3772,missing,missing,25.0833,0,na,1,na,1,0,missing,29.29,0,1,1.00000,na,0,C,89,795,4,3180,1.0
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,missing,0,0,84235,missing,0,0,0,0,0,NaN,0.00,NaN,0.00,1308,2525,791,91,5992,missing,3741,215,10.3333,0,0,3,3,2,0,96.272,missing,0.15385,3,0.53241,0,0,L,3,755,4,3020,0.0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,missing,2257,27815,123875,524848,0,0,0,0,0,20.51,796.67,NaN,28.72,801,2281,2281,487,2707,missing,1947,158,0.9167,0,0,2,3,2,0,115.019,missing,0,1,0.92665,na,na,L,5,425,4,1700,0.0
5,230037,1579,NaN,3.502,0.0,0.0,missing,missing,missing,missing,0,199,missing,missing,77298,missing,na,na,na,0,0,NaN,NaN,NaN,NaN,missing,missing,missing,183,973,missing,missing,missing,1.3333,0,na,2,na,2,na,missing,missing,1.5,0,NaN,na,na,C,35,455,4,1820,1.0
6,230038,1818,0.4001,0.000,0.0,0.0,missing,1088,missing,1536,1498,missing,448,2331,123875,missing,na,0,0,0,0,NaN,NaN,NaN,29.16,1034,760,missing,30386,3437,missing,791,34,9.3333,0,na,2,0,0,0,88.171,missing,0,2,0.87224,na,0,C,2,392,5,1960,1.0
7,230039,na,NaN,NaN,NaN,NaN,missing,missing,missing,missing,missing,missing,missing,missing,198200,1263525,na,na,na,0,na,NaN,NaN,NaN,NaN,missing,missing,missing,missing,missing,missing,missing,missing,0.0833,na,na,na,na,na,na,missing,missing,missing,na,NaN,na,na,C,2,615,#VALUE!,#VALUE!,0.0
8,230040,1836,0.1358,0.000,0.0,0.0,347,38964,17828,70729,65843,missing,16298,18877,109010,missing,0,0,0,1,1,32.02,0.00,0.00,

In [75]:
df.replace('missing', np.nan, inplace = True)
df.replace('na', np.nan, inplace = True)
df.replace('#VALUE!', np.nan, inplace = True)

In [76]:
df.head(10)

,ID,Credit Worthiness Score,Number and Riskiness of Credit Enquiry,Default Severity (Any Loan),Default Severity (Auto Loan),Default Severity (Education Loan),Min Cred Available (Revolving Credit Cards),Max Cred Available (Active Credit Lines),Max Cred Available (Active Revolving Credit Cards),Total Available Credit (Cards with 1 missed payment),Total Credit Available,Dues Collected Post Default,Total Amount Due (Active Cards),Credit Amount Paid (Prev Year),Annual Income,Property Value,Full Credit Utilisation (Revolving Credit Cards),Full Credit Utilisation (Credit Cards),Full Credit Utilisation (Credit Lines),>75 percent Credit Utilisation (Credit Cards),>75 percent Credit Utilisation (Credit Lines),Average Utilisation (Revolving Credit Cards),Average Utilisation Last 2 Years (All Credit Lines),Average Utilisation Last Year (All Credit Cards),Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards),Average Tenure (Active Revolving Credit Cards),Tenure of Oldest Active Credit Card,Tenure of Oldest Active Revolving Credit Card,Days Since Last Missed Payment,Tenure of Oldest Credit Line,Max Tenure (Auto Loans),Max Tenure (Education Loans),Sum of Tenures (Active Credit Cards),Stay Duration of Current Residence,Credit Lines with 1 Missed Payment (Last 6 Months),Revolving Credit Cards with 1 Missed Payment (Last 2 Years),Active Credit Lines,Credit Cards with >2Y Tenure,Credit Lines activated Prev 2 Years,Credit Lines with Current Delinquency,Utilization on Active Education Loans,Utilization on Active Auto Loans,Financial Stress Index,Credit Lines with No Missed Payments (Prev 2 Years),Ratio(Max Amount Due : Sum of Amounts Due),Mortgage Loans (With 2 Missed Payements),Auto Loans (With 2 Missed Payments),Type of Product,Int Value for an Application,Bucketized Credit Worthiness Score,Compound of mvar49 and mvar48,mvar51,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,NaN,6423,3067,123875,NaN,1,1,1,3,3,94.78,8987.18,NaN,72.25,1462,4532,2890,61,4532,1095,3376,625,1.1667,1,0,8,10,4,1,73.78,82.547,0.08696,10,0.63899,NaN,0,C,10,770,4,3080,0.0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,NaN,765,1931,42613,NaN,0,0,0,0,0,74.25,953.06,953.06,4.80,1028,2099,2099,30386,2281,NaN,2251,169,0.4167,0,0,8,0,2,0,99.129,NaN,0,13,0.63836,NaN,NaN,L,732,437,5,2185,1.0
2,230034,1745,0.4001,0.000,0.0,0.0,NaN,2536,NaN,NaN,2536,NaN,NaN,NaN,76109,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,669,4623,3772,NaN,NaN,25.0833,0,NaN,1,NaN,1,0,NaN,29.29,0,1,1.00000,NaN,0,C,89,795,4,3180,1.0
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,NaN,0,0,84235,NaN,0,0,0,0,0,NaN,0.00,NaN,0.00,1308,2525,791,91,5992,NaN,3741,215,10.3333,0,0,3,3,2,0,96.272,NaN,0.15385,3,0.53241,0,0,L,3,755,4,3020,0.0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,NaN,2257,27815,123875,524848,0,0,0,0,0,20.51,796.67,NaN,28.72,801,2281,2281,487,2707,NaN,1947,158,0.9167,0,0,2,3,2,0,115.019,NaN,0,1,0.92665,NaN,NaN,L,5,425,4,1700,0.0
5,230037,1579,NaN,3.502,0.0,0.0,NaN,NaN,NaN,NaN,0,199,NaN,NaN,77298,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183,973,NaN,NaN,NaN,1.3333,0,NaN,2,NaN,2,NaN,NaN,NaN,1.5,0,NaN,NaN,NaN,C,35,455,4,1820,1.0
6,230038,1818,0.4001,0.000,0.0,0.0,NaN,1088,NaN,1536,1498,NaN,448,2331,123875,NaN,NaN,0,0,0,0,NaN,NaN,NaN,29.16,1034,760,NaN,30386,3437,NaN,791,34,9.3333,0,NaN,2,0,0,0,88.171,NaN,0,2,0.87224,NaN,0,C,2,392,5,1960,1.0
7,230039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198200,1263525,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,2,615,NaN,NaN,0.0
8,230040,1836,0.1358,0.000,0.0,0.0,347,38964,17828,70729,65843,NaN,16298,18877,109010,NaN,0,0,0,1,1,32.02,0.00,0.00,32.92,3180,8943,8943,1490,12075,NaN,NaN,1673,5.9167,0,NaN,6,12,2,0,NaN,NaN,0,10,0.89868,0,0,L,5,821,5,4105,1.0
9,230041,1839,0.1981,0.000,0.0,0.0,793,6131,6045,48959,31640,NaN,10565,14282,99100,146668,0,0,0,1,1,46.17,NaN,NaN,49.07,1701,8912,2707,61

In [77]:
df.reset_index(drop = True, inplace = True)

In [78]:
def convert_to_int(entry):
    return float(entry)

for col in df.columns:
    if col != 'Type of Product':
        df[col] = df[col].apply(convert_to_int)
    
df['Credit Worthiness Score'][0]

1696.0

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  float64
 1   Credit Worthiness Score                                                 124039 non-null  float64
 2   Number and Riskiness of Credit Enquiry                                  120787 non-null  float64
 3   Default Severity (Any Loan)                                             129171 non-null  float64
 4   Default Severity (Auto Loan)                                            129171 non-null  float64
 5   Default Severity (Education Loan)                                       129171 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                         

In [80]:
le = LabelEncoder()
df['Type of Product'] = le.fit_transform(df['Type of Product'])

In [81]:
df.columns

Index(['ID', 'Credit Worthiness Score',
       'Number and Riskiness of Credit Enquiry', 'Default Severity (Any Loan)',
       'Default Severity (Auto Loan)', 'Default Severity (Education Loan)',
       'Min Cred Available (Revolving Credit Cards)',
       'Max Cred Available (Active Credit Lines)',
       'Max Cred Available (Active Revolving Credit Cards)',
       'Total Available Credit (Cards with 1 missed payment)',
       'Total Credit Available', 'Dues Collected Post Default',
       'Total Amount Due (Active Cards)', 'Credit Amount Paid (Prev Year)',
       'Annual Income', 'Property Value',
       'Full Credit Utilisation (Revolving Credit Cards)',
       'Full Credit Utilisation (Credit Cards)',
       'Full Credit Utilisation (Credit Lines)',
       '>75 percent Credit Utilisation (Credit Cards)',
       '>75 percent Credit Utilisation (Credit Lines)',
       'Average Utilisation (Revolving Credit Cards)',
       'Average Utilisation Last 2 Years (All Credit Lines)',
       

In [82]:
cols_to_scale = ['Credit Worthiness Score',
       'Number and Riskiness of Credit Enquiry', 'Default Severity (Any Loan)',
       'Default Severity (Auto Loan)', 'Default Severity (Education Loan)',
       'Min Cred Available (Revolving Credit Cards)',
       'Max Cred Available (Active Credit Lines)',
       'Max Cred Available (Active Revolving Credit Cards)',
       'Total Available Credit (Cards with 1 missed payment)',
       'Total Credit Available', 'Dues Collected Post Default',
       'Total Amount Due (Active Cards)', 'Credit Amount Paid (Prev Year)',
       'Annual Income', 'Property Value',
       'Full Credit Utilisation (Revolving Credit Cards)',
       'Full Credit Utilisation (Credit Cards)',
       'Full Credit Utilisation (Credit Lines)',
       '>75 percent Credit Utilisation (Credit Cards)',
       '>75 percent Credit Utilisation (Credit Lines)',
       'Average Utilisation (Revolving Credit Cards)',
       'Average Utilisation Last 2 Years (All Credit Lines)',
       'Average Utilisation Last Year (All Credit Cards)',
       'Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards)',
       'Average Tenure (Active Revolving Credit Cards)',
       'Tenure of Oldest Active Credit Card',
       'Tenure of Oldest Active Revolving Credit Card',
       'Days Since Last Missed Payment', 'Tenure of Oldest Credit Line',
       'Max Tenure (Auto Loans)', 'Max Tenure (Education Loans)',
       'Sum of Tenures (Active Credit Cards)',
       'Stay Duration of Current Residence',
       'Credit Lines with 1 Missed Payment (Last 6 Months)',
       'Revolving Credit Cards with 1 Missed Payment (Last 2 Years)',
       'Active Credit Lines', 'Credit Cards with >2Y Tenure',
       'Credit Lines activated Prev 2 Years',
       'Credit Lines with Current Delinquency',
       'Utilization on Active Education Loans',
       'Utilization on Active Auto Loans', 'Financial Stress Index',
       'Credit Lines with No Missed Payments (Prev 2 Years)',
       'Ratio(Max Amount Due : Sum of Amounts Due)',
       'Mortgage Loans (With 2 Missed Payements)',
       'Auto Loans (With 2 Missed Payments)',
       'Int Value for an Application', 'Bucketized Credit Worthiness Score',
       'Compound of mvar49 and mvar48', 'mvar51']
df[cols_to_scale] = df[cols_to_scale] + 0.01
df[cols_to_scale] = df[cols_to_scale].apply(np.log, axis = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  float64
 1   Credit Worthiness Score                                                 124039 non-null  float64
 2   Number and Riskiness of Credit Enquiry                                  120787 non-null  float64
 3   Default Severity (Any Loan)                                             129171 non-null  float64
 4   Default Severity (Auto Loan)                                            129171 non-null  float64
 5   Default Severity (Education Loan)                                       129171 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                         

In [83]:
df.head()

,ID,Credit Worthiness Score,Number and Riskiness of Credit Enquiry,Default Severity (Any Loan),Default Severity (Auto Loan),Default Severity (Education Loan),Min Cred Available (Revolving Credit Cards),Max Cred Available (Active Credit Lines),Max Cred Available (Active Revolving Credit Cards),Total Available Credit (Cards with 1 missed payment),Total Credit Available,Dues Collected Post Default,Total Amount Due (Active Cards),Credit Amount Paid (Prev Year),Annual Income,Property Value,Full Credit Utilisation (Revolving Credit Cards),Full Credit Utilisation (Credit Cards),Full Credit Utilisation (Credit Lines),>75 percent Credit Utilisation (Credit Cards),>75 percent Credit Utilisation (Credit Lines),Average Utilisation (Revolving Credit Cards),Average Utilisation Last 2 Years (All Credit Lines),Average Utilisation Last Year (All Credit Cards),Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards),Average Tenure (Active Revolving Credit Cards),Tenure of Oldest Active Credit Card,Tenure of Oldest Active Revolving Credit Card,Days Since Last Missed Payment,Tenure of Oldest Credit Line,Max Tenure (Auto Loans),Max Tenure (Education Loans),Sum of Tenures (Active Credit Cards),Stay Duration of Current Residence,Credit Lines with 1 Missed Payment (Last 6 Months),Revolving Credit Cards with 1 Missed Payment (Last 2 Years),Active Credit Lines,Credit Cards with >2Y Tenure,Credit Lines activated Prev 2 Years,Credit Lines with Current Delinquency,Utilization on Active Education Loans,Utilization on Active Auto Loans,Financial Stress Index,Credit Lines with No Missed Payments (Prev 2 Years),Ratio(Max Amount Due : Sum of Amounts Due),Mortgage Loans (With 2 Missed Payements),Auto Loans (With 2 Missed Payments),Type of Product,Int Value for an Application,Bucketized Credit Worthiness Score,Compound of mvar49 and mvar48,mvar51,default_ind
0,230032.0,7.436034,0.509284,-4.60517,-4.60517,-4.60517,-4.605170,8.702013,5.774583,10.605818,9.820867,NaN,8.767642,8.028458,11.727028,NaN,0.00995,0.00995,0.00995,1.10194,1.10194,4.551664,9.103556,NaN,4.280271,7.287567,8.418921,7.969015,4.111038,8.418921,6.998519,8.124450,6.437768,0.162714,0.00995,-4.60517,2.080691,2.303585,1.388791,0.00995,4.301223,4.413489,-2.333457,2.303585,-0.432338,NaN,-4.60517,0,2.303585,6.646404,1.388791,8.032688,0.0
1,230033.0,7.520782,-0.199061,-4.60517,-4.60517,-4.60517,4.625071,8.926917,8.061805,9.811044,9.522521,NaN,6.639889,7.565798,10.659915,NaN,-4.60517,-4.60517,-4.60517,-4.60517,-4.60517,4.307572,6.859688,6.859688,1.570697,6.935380,7.649221,7.649221,10.321738,7.732374,NaN,7.719134,5.129958,-0.851674,-4.60517,-4.60517,2.080691,-4.605170,0.698135,-4.60517,4.596523,NaN,-4.605170,2.565718,-0.433309,NaN,NaN,1,6.595794,6.079956,1.611436,7.689376,1.0
2,230034.0,7.464516,-0.891354,-4.60517,-4.60517,-4.60517,NaN,7.838347,NaN,NaN,7.838347,NaN,NaN,NaN,11.239922,NaN,NaN,NaN,NaN,-4.60517,-4.60517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.505799,8.438801,8.235363,NaN,NaN,3.222601,-4.60517,NaN,0.009950,NaN,0.009950,-4.60517,NaN,3.377588,-4.605170,0.009950,0.009950,NaN,-4.60517,0,4.488749,6.678355,1.388791,8.064640,1.0
3,230035.0,7.461071,-1.472724,-4.60517,-4.60517,-4.60517,7.591867,10.182634,8.508154,9.919165,10.519025,NaN,-4.605170,-4.605170,11.341366,NaN,-4.60517,-4.60517,-4.60517,-4.60517,-4.60517,NaN,-4.605170,NaN,-4.605170,7.176262,7.834000,6.673311,4.510969,8.698182,NaN,8.227111,5.370685,2.336339,-4.60517,-4.60517,1.101940,1.101940,0.698135,-4.60517,4.567281,NaN,-1.808804,1.101940,-0.611733,-4.60517,-4.60517,1,1.101940,6.626731,1.388791,8.013015,0.0
4,230036.0,7.488299,-3.825845,-1.44817,-4.60517,-4.60517,8.603556,8.611414,8.611414,8.985572,8.454468,NaN,7.721796,10.233331,11.727028,13.170864,-4.60517,-4.60517,-4.60517,-4.60517,-4.60517,3.021400,6.680453,NaN,3.357942,6.685873,7.732374,7.732374,6.188285,7.903600,NaN,7.574050,5.062658,-0.076125,-4.60517,-4.60517,0.698135,1.101940,0.698135,-4.60517,4.745184,NaN,-4.605170,0.009950,-0.065446,NaN,NaN,1,1.611436,6.052113,1.388791,7.438389,

In [84]:
sc = StandardScaler()
df[cols_to_scale] = pd.DataFrame(sc.fit_transform(df[cols_to_scale]), columns = cols_to_scale)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  float64
 1   Credit Worthiness Score                                                 124039 non-null  float64
 2   Number and Riskiness of Credit Enquiry                                  120787 non-null  float64
 3   Default Severity (Any Loan)                                             129171 non-null  float64
 4   Default Severity (Auto Loan)                                            129171 non-null  float64
 5   Default Severity (Education Loan)                                       129171 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                         

In [85]:
df.fillna(-1, inplace = True)

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  float64
 1   Credit Worthiness Score                                                 130000 non-null  float64
 2   Number and Riskiness of Credit Enquiry                                  130000 non-null  float64
 3   Default Severity (Any Loan)                                             130000 non-null  float64
 4   Default Severity (Auto Loan)                                            130000 non-null  float64
 5   Default Severity (Education Loan)                                       130000 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                         

In [87]:
df['default_ind'].value_counts()

 0.0    59145
-1.0    47000
 1.0    23855
Name: default_ind, dtype: int64

In [88]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [89]:
train = df.iloc[:83000, :]

X_data = train.drop(['ID', 'default_ind'], axis = 1)
y_data = train['default_ind']

In [138]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X_data, y_data, test_size = 0.15, stratify = y_data, random_state=0)

nn = MLPClassifier(hidden_layer_sizes=(8, 8, 8, 8, 8), learning_rate='adaptive', random_state=0, verbose = True,
                   early_stopping=True, validation_fraction=0.15, max_iter = 5000, learning_rate_init=1e-5,
                   n_iter_no_change=50, tol = 5e-5, solver = 'lbfgs')
nn.fit(X_Train, Y_Train)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          705     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.07033D-01    |proj g|=  2.03457D-02

At iterate    1    f=  6.02585D-01    |proj g|=  8.88633D-02

At iterate    2    f=  5.58970D-01    |proj g|=  3.35729D-02


 This problem is unconstrained.



At iterate    3    f=  5.40396D-01    |proj g|=  1.51042D-02

At iterate    4    f=  5.30521D-01    |proj g|=  1.43698D-02

At iterate    5    f=  5.21569D-01    |proj g|=  2.74439D-02

At iterate    6    f=  5.18922D-01    |proj g|=  4.14709D-02

At iterate    7    f=  5.15845D-01    |proj g|=  6.01970D-03

At iterate    8    f=  5.15081D-01    |proj g|=  8.05885D-03

At iterate    9    f=  5.13529D-01    |proj g|=  6.59878D-03

At iterate   10    f=  5.09711D-01    |proj g|=  1.09850D-02

At iterate   11    f=  5.06834D-01    |proj g|=  2.93239D-02

At iterate   12    f=  5.06138D-01    |proj g|=  1.12179D-02

At iterate   13    f=  5.05613D-01    |proj g|=  4.33265D-03

At iterate   14    f=  5.05431D-01    |proj g|=  3.41426D-03

At iterate   15    f=  5.04811D-01    |proj g|=  4.17788D-03

At iterate   16    f=  5.03492D-01    |proj g|=  1.91193D-02

At iterate   17    f=  5.02315D-01    |proj g|=  1.07636D-01

At iterate   18    f=  5.01255D-01    |proj g|=  4.06666D-02

At iter


At iterate  137    f=  4.91938D-01    |proj g|=  1.96546D-02

At iterate  138    f=  4.91931D-01    |proj g|=  2.63295D-03

At iterate  139    f=  4.91928D-01    |proj g|=  2.65496D-03

At iterate  140    f=  4.91917D-01    |proj g|=  7.65406D-03

At iterate  141    f=  4.91911D-01    |proj g|=  5.22688D-03

At iterate  142    f=  4.91872D-01    |proj g|=  7.28532D-03

At iterate  143    f=  4.91833D-01    |proj g|=  1.26587D-01

At iterate  144    f=  4.91716D-01    |proj g|=  6.59320D-03

At iterate  145    f=  4.91659D-01    |proj g|=  2.04941D-02

At iterate  146    f=  4.91494D-01    |proj g|=  2.94887D-02

At iterate  147    f=  4.91480D-01    |proj g|=  2.80790D-02

At iterate  148    f=  4.91342D-01    |proj g|=  1.04475D-01

At iterate  149    f=  4.91190D-01    |proj g|=  3.80937D-02

At iterate  150    f=  4.90985D-01    |proj g|=  5.04468D-02

At iterate  151    f=  4.90896D-01    |proj g|=  5.26242D-02

At iterate  152    f=  4.90815D-01    |proj g|=  2.86625D-03

At iter


At iterate  271    f=  4.89065D-01    |proj g|=  1.87965D-02

At iterate  272    f=  4.89063D-01    |proj g|=  9.40193D-03

At iterate  273    f=  4.89060D-01    |proj g|=  1.53735D-02

At iterate  274    f=  4.89057D-01    |proj g|=  1.86985D-03

At iterate  275    f=  4.89056D-01    |proj g|=  1.52403D-02

At iterate  276    f=  4.89054D-01    |proj g|=  1.21837D-02

At iterate  277    f=  4.89051D-01    |proj g|=  3.36572D-03

At iterate  278    f=  4.89045D-01    |proj g|=  8.59202D-03

At iterate  279    f=  4.89031D-01    |proj g|=  1.35365D-02

At iterate  280    f=  4.89026D-01    |proj g|=  5.91585D-03

At iterate  281    f=  4.89016D-01    |proj g|=  1.21748D-02

At iterate  282    f=  4.88994D-01    |proj g|=  2.52441D-02

At iterate  283    f=  4.88965D-01    |proj g|=  2.07521D-02

At iterate  284    f=  4.88946D-01    |proj g|=  5.41317D-03

At iterate  285    f=  4.88925D-01    |proj g|=  1.23416D-02

At iterate  286    f=  4.88916D-01    |proj g|=  2.53648D-02

At iter

...

At iterate 4561    f=  4.75304D-01    |proj g|=  6.20982D-03

At iterate 4562    f=  4.75304D-01    |proj g|=  3.10279D-02

At iterate 4563    f=  4.75301D-01    |proj g|=  2.80276D-02

At iterate 4564    f=  4.75301D-01    |proj g|=  6.85264D-02

At iterate 4565    f=  4.75300D-01    |proj g|=  4.52429D-02

At iterate 4566    f=  4.75300D-01    |proj g|=  2.52085D-02

At iterate 4567    f=  4.75299D-01    |proj g|=  2.55563D-03

At iterate 4568    f=  4.75299D-01    |proj g|=  6.43058D-03

At iterate 4569    f=  4.75299D-01    |proj g|=  1.99258D-02

At iterate 4570    f=  4.75299D-01    |proj g|=  1.35438D-02

At iterate 4571    f=  4.75299D-01    |proj g|=  4.80605D-03

At iterate 4572    f=  4.75298D-01    |proj g|=  4.89189D-02

At iterate 4573    f=  4.75298D-01    |proj g|=  8.01109D-03

At iterate 4574    f=  4.75298D-01    |proj g|=  4.42969D-03

At iterate 4575    f=  4.75298D-01    |proj g|=  1.01964D-02

At iterate 4576    f=  4.75298D-01    |proj g|=  2.24694D-02

At itera


At iterate 4695    f=  4.75241D-01    |proj g|=  3.57892D-02

At iterate 4696    f=  4.75241D-01    |proj g|=  4.09377D-02

At iterate 4697    f=  4.75240D-01    |proj g|=  2.83596D-02

At iterate 4698    f=  4.75239D-01    |proj g|=  3.89025D-02

At iterate 4699    f=  4.75238D-01    |proj g|=  2.40669D-02

At iterate 4700    f=  4.75238D-01    |proj g|=  1.64964D-02

At iterate 4701    f=  4.75238D-01    |proj g|=  6.26522D-03

At iterate 4702    f=  4.75237D-01    |proj g|=  6.60390D-03

At iterate 4703    f=  4.75237D-01    |proj g|=  1.99995D-02

At iterate 4704    f=  4.75236D-01    |proj g|=  2.90392D-02

At iterate 4705    f=  4.75235D-01    |proj g|=  1.39937D-02

At iterate 4706    f=  4.75234D-01    |proj g|=  1.12075D-02

At iterate 4707    f=  4.75233D-01    |proj g|=  1.12275D-02

At iterate 4708    f=  4.75233D-01    |proj g|=  2.68619D-02

At iterate 4709    f=  4.75232D-01    |proj g|=  1.65878D-02

At iterate 4710    f=  4.75231D-01    |proj g|=  2.23883D-02

At iter


At iterate 4828    f=  4.75172D-01    |proj g|=  4.07478D-03

At iterate 4829    f=  4.75172D-01    |proj g|=  1.21832D-02

At iterate 4830    f=  4.75172D-01    |proj g|=  3.84174D-02

At iterate 4831    f=  4.75171D-01    |proj g|=  1.96223D-02

At iterate 4832    f=  4.75171D-01    |proj g|=  3.98522D-03

At iterate 4833    f=  4.75171D-01    |proj g|=  6.26756D-03

At iterate 4834    f=  4.75171D-01    |proj g|=  6.90339D-03

At iterate 4835    f=  4.75171D-01    |proj g|=  2.91364D-02

At iterate 4836    f=  4.75170D-01    |proj g|=  1.71816D-02

At iterate 4837    f=  4.75170D-01    |proj g|=  1.49662D-02

At iterate 4838    f=  4.75170D-01    |proj g|=  2.00132D-02

At iterate 4839    f=  4.75170D-01    |proj g|=  9.30175D-03

At iterate 4840    f=  4.75169D-01    |proj g|=  1.45434D-02

At iterate 4841    f=  4.75169D-01    |proj g|=  9.67149D-03

At iterate 4842    f=  4.75169D-01    |proj g|=  4.35727D-02

At iterate 4843    f=  4.75169D-01    |proj g|=  3.72491D-02

At iter


At iterate 4963    f=  4.75139D-01    |proj g|=  5.51513D-02

At iterate 4964    f=  4.75139D-01    |proj g|=  2.89390D-02

At iterate 4965    f=  4.75138D-01    |proj g|=  2.14847D-03

At iterate 4966    f=  4.75138D-01    |proj g|=  1.52138D-02

At iterate 4967    f=  4.75138D-01    |proj g|=  1.51384D-02

At iterate 4968    f=  4.75137D-01    |proj g|=  2.15403D-02

At iterate 4969    f=  4.75137D-01    |proj g|=  2.27583D-02

At iterate 4970    f=  4.75137D-01    |proj g|=  2.60775D-02

At iterate 4971    f=  4.75137D-01    |proj g|=  2.18797D-02

At iterate 4972    f=  4.75136D-01    |proj g|=  3.25354D-02

At iterate 4973    f=  4.75136D-01    |proj g|=  3.19481D-02

At iterate 4974    f=  4.75136D-01    |proj g|=  1.56629D-02

At iterate 4975    f=  4.75136D-01    |proj g|=  1.56418D-02

At iterate 4976    f=  4.75136D-01    |proj g|=  5.65157D-03

At iterate 4977    f=  4.75135D-01    |proj g|=  7.22959D-03

At iterate 4978    f=  4.75134D-01    |proj g|=  2.55385D-02

At iter

MLPClassifier(early_stopping=True, hidden_layer_sizes=(8, 8, 8, 8, 8),
              learning_rate='adaptive', learning_rate_init=1e-05, max_iter=5000,
              n_iter_no_change=50, random_state=0, solver='lbfgs', tol=5e-05,
              validation_fraction=0.15, verbose=True)

In [139]:
nn.loss_

0.47512703664283373

In [140]:
preds = nn.predict(X_Test)

print("Accuracy Score: ", accuracy_score(preds, Y_Test)*100)
print("F1 Score: ", f1_score(preds, Y_Test))

Accuracy Score:  76.79518072289156
F1 Score:  0.5077525983983644


At this point I've just encountered a huge WTF moment with Sklearn's MLP Classifier. WTH is it even calculating early stopping on. Not just that, there is legit no minima or maxima at the point where it claims to have reached an optimum. This is completely useless and was a waste of time. I'm shifting to Pytorch, F this.

In [48]:
test = df.iloc[83000:, :]
test = test.drop(['ID', 'default_ind'], axis = 1)

In [ ]:
#Logic for Prediction

In [56]:
sample = pd.read_csv('submission.csv', header = None)
sample.head()

,0,1
0,578069,0
1,578070,1
2,578071,1
3,578072,1
4,578073,0


In [57]:
sample[1] = preds
sample.head()

,0,1
0,578069,0
1,578070,0
2,578071,0
3,578072,0
4,578073,0


In [58]:
sample[1].value_counts()

0    34808
1    12192
Name: 1, dtype: int64

In [59]:
#sample.to_csv('Submissions/ThirdDegreeBurn_5.csv', index = False, header=False)